In [1]:
%load_ext autoreload
%autoreload 2
    
import os
import sys
import yaml
import json
import base64
from collections import defaultdict
from typing import Tuple
import functools

sys.path.append('/home/lishengping/projects/maxtext/MaxText')
os.environ['HARDWARE'] = 'tpu'

from layers import models
import max_utils
import jax
import orbax
import jax.numpy as jnp
from jax.sharding import Mesh
from flax.traverse_util import flatten_dict, unflatten_dict
from flax import linen as nn
from transformers import AutoTokenizer
from etils import epath

import pyconfig
from jax.sharding import PartitionSpec
from flax.linen import partitioning as nn_partitioning


TOKENIZER_PATH = '/home/lishengping/tokenizer'
if not os.path.exists(TOKENIZER_PATH):
    !gsutil cp -r gs://llm_base_models_us-east5/qwen/tokenizer /home/lishengping/
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH, use_fast=True, trust_remote_code=True)

read_dir = "gs://llm_base_models_europe-west4/v5p_256/7B/xm_E8T2S0x7B_WithMgate_openmoe_RouterR_1218/checkpoints/"
read_dir = epath.Path(read_dir)

# config_name = '/home/lishengping/projects/maxtext/MaxText/configs/dcformer_pp_405m.yml'
config_name = '/home/lishengping/projects/maxtext/MaxText/configs/dc_forward.yml'

argv = [None, config_name]
pyconfig.initialize(argv)
config = pyconfig.config
# validate_train_config(config)
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)

2024-12-18 07:50:18.994389: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-18 07:50:19.007909: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-18 07:50:19.007932: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-18 07:50:19.656298: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Updating keys from env and command line: []
Running Model: default
Updating keys from model: []
System Information: Jax Version: 0.4.35
System Information: Jaxlib Version: 0.4.35
System Information: Jax Backend: PJRT C API
TFRT TPU v6 lite
Built on Oct 21 2024 00:24:02 (1729495442) cl/687888698
Not using emergency checkpoint, ignoring local_checkpoint_directory and local_checkpoint_period
Config param activations_in_float32: False
Config param adam_b1: 0.9
Config param adam_b2: 0.95
Config param adam_eps: 1e-08
Config param adam_eps_root: 0.0
Config param adam_weight_decay: 0.1
Config param allow_split_physical_axes: False
Config param ar_cache_axis_order: 1,2,0,3
Config param async_checkpointing: True
Config param attention: dot_product
Config param autoregressive_decode_assert: 
Config param aux_loss_coef: 0.0
Config param base_emb_dim: 4096
Config param base_mlp_dim: 5632
Config param base_num_decoder_layers: 48
Config param base_num_kv_heads: 32
Config param base_num_query_heads: 3

In [2]:
def decode_base64(encoded_str):
    decoded_bytes = base64.b64decode(encoded_str)
    decoded_str = decoded_bytes.decode('utf-8')
    return decoded_str


def mesh_shard_rules(mesh, rules, remove_keys=[]):
    _sharding_dict = {}
    for name, rule in rules.items():
        if isinstance(rule, str):
            rule = json.loads(rule)
        name = decode_base64(name)
        param_key = tuple(name.split('.'))
        remove = any([1 if key in param_key else 0 for key in remove_keys])
        if remove: continue
        prule = [tuple(r) if isinstance(r, list) else r for r in rule['partition_spec'] ]
        spec = jax.sharding.PartitionSpec(*prule)
        _sharding_dict[param_key] = jax.sharding.NamedSharding(mesh, spec)
    return _sharding_dict


def rewrite_bucket_sharding(mesh, old_sharding, save_path):
    cur_machine_sharding = {}
    for k, v in old_sharding.items():
        if isinstance(v, str):
            v = json.loads(v)
        v['shape'] = mesh.device_ids.shape
        cur_machine_sharding[k] = v
    save_path = epath.Path(save_path)
    with save_path.open('w') as f:
        json.dump(cur_machine_sharding, f)
    
load_step = 0
_sharding_path = epath.Path('gs://llm_base_models_europe-west4/v5p_256/7B/xm3p5_7b_with_perlayer_moe_sharding')
_metadata_path = read_dir / str(load_step) / 'state/_METADATA'

_metadata_path = read_dir / str(load_step) / 'state/_METADATA'

# delete file or dir
# _sharding_path.unlink()

remove_keys = ['opt_state', 'step']
if _sharding_path.exists():
    with _sharding_path.open('r') as f:
        _sharding_rules = json.load(f)
    # 重写_sharding文件
    rewrite_bucket_sharding(mesh, _sharding_rules, _sharding_path)
    _sharding_dict = mesh_shard_rules(mesh, _sharding_rules, remove_keys=remove_keys)
    _sharding_dict = unflatten_dict(_sharding_dict)
elif _metadata_path.exists():
    _metadata_dict = {}
    with _metadata_path.open('r') as f:
        _metadata = json.load(f)
    for param_key in _metadata['tree_metadata']:
        if isinstance(param_key, str): param_key = eval(param_key)
        remove = any([1 if key in param_key else 0 for key in remove_keys])
        if remove: continue
        _metadata_dict[param_key] = jax.ShapeDtypeStruct(shape=(), dtype=jnp.float32)
    _metadata_dict = unflatten_dict(_metadata_dict)
    
else:
    _sharding_dict = None
    _metadata_dict = None

In [7]:
# !pip install tensorflow==2.16.1
# pip install numpy==1.26.4
# 运行2遍
import json
import os
import sys
import asyncio
import argparse
from collections import defaultdict
import time

# os.environ["JAX_PLATFORMS"] = "cpu"
from etils import epath
import json
import base64

import torch
import numpy as np
import jax.numpy as jnp
import jax
import orbax
import orbax.checkpoint as ocp
from etils import epath
from jax.sharding import PartitionSpec as PS
from flax.traverse_util import flatten_dict, unflatten_dict

metadata_path = _metadata_path
METADATA_FILE = '_METADATA'
_CHECKPOINT_FILE = 'checkpoint'

read_dir = "gs://llm_base_models_europe-west4/v5p_256/7B/xm_E8T2S0x7B_WithMgate_openmoe_RouterR_1218/checkpoints/0/state"
read_dir = epath.Path(read_dir)

back_metadata_path = read_dir / f'{METADATA_FILE}.back'
try:
    metadata_path.rename(back_metadata_path)
except:
    pass
metadata_path.unlink(missing_ok=True) # delete
structure_path = read_dir / _CHECKPOINT_FILE
msgpack = ocp.aggregate_handlers.MsgpackHandler(0)
structure = msgpack.deserialize(structure_path)
# backup original checkpoint fil
back_structure_path = read_dir / 'checkpoint_back'
back_structure = structure.copy()
if not back_structure_path.exists():
    asyncio.run(msgpack.serialize(back_structure_path, item=back_structure))
print(f'Old structure file keys: {structure.keys()}')
remove_keys = ['opt_state', 'step'] # select the weight name you don't want to load, all weight name: opt_state, step, params
_ = [structure.pop(key) for key in remove_keys if key in structure]
print(f'New structure file keys: {structure.keys()}')
asyncio.run(msgpack.serialize(structure_path, item=structure))  # rewrite struct file

# load model based struct, note: axes must same as training
mesh_axes = ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor', 'expert', 'autoregressive']
axes = [1] * len(mesh_axes)
axes[2] = 4
devices = np.asarray(jax.devices()).reshape(axes)
mesh = jax.sharding.Mesh(devices, mesh_axes)
sharding = jax.sharding.NamedSharding(mesh, PS()) # Sharding is None because we use cpu to load weights
weight_dtype = jnp.bfloat16 # set restore weights dtype
restore_args = {}
for k, v in flatten_dict(structure).items():
    restore_args[k] =  ocp.ArrayRestoreArgs(restore_type=jax.Array, dtype=weight_dtype, sharding=sharding)
restore_args = unflatten_dict(restore_args)

# 新加的
right_shard = flatten_dict(_sharding_dict)
new_restore_args = {}
for k, v in flatten_dict(restore_args).items():
    print(k, v)
    v.sharding = right_shard[k]
    new_restore_args[k] = v
restore_args = unflatten_dict(new_restore_args)

ckptr = ocp.Checkpointer(ocp.PyTreeCheckpointHandler())
params = ckptr.restore(read_dir, args=ocp.args.PyTreeRestore(restore_args=restore_args))
structure_path = read_dir / _CHECKPOINT_FILE
# rewrite struct file, otherwise occur error when continue training
asyncio.run(msgpack.serialize(structure_path, item=back_structure))
while 'params' in w:
    params = params['params']
# xm3p5_w = {'.'.join(k): np.array(v) for k, v in flatten_dict(w).items()}
try:
    back_metadata_path.rename(metadata_path)
except:
    pass

In [8]:
for k, v in flatten_dict(w).items():
    print(k, v.devices())
    break

('params', 'params', 'decoder', 'decoder_norm', 'scale') {TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)}


In [9]:
quant = None

Transformer = models.Transformer
model = Transformer(config, mesh, quant=quant)
is_train = False
rng1, aqt_rng = jax.random.split(jax.random.key(9876))

In [10]:
text = '<|extra_0|>飞流直下三千尺，疑是银河落九天。'
x = tokenizer(text, return_tensor='pt')
batch_size = 4
input_ids = jnp.array(x['input_ids']).reshape(1, -1).repeat(batch_size, 0)
data = {}
data['inputs'] = input_ids[:, :-1]
pos = jnp.arange(data['inputs'].shape[1]).reshape(1, -1).repeat(batch_size, 0)
data["inputs_position"] = jnp.broadcast_to(pos, (batch_size, pos.shape[-1]))
data["inputs_segmentation"] = jnp.ones_like(data['inputs'])
data["targets"] = input_ids[:, 1:]
data = {k: v[:, :] for k, v in data.items()}
print(f'数据制作完成.')

数据制作完成.


In [14]:
logits, intermediate_outputs = model.apply(
          {'params':params},
          data["inputs"],
          data["inputs_position"],
          decoder_segment_ids=data["inputs_segmentation"],
          enable_dropout=config.enable_dropout if is_train else False,
          rngs={"dropout": rng1, "params": aqt_rng},
          mutable="intermediates",
      )
one_hot_targets = jax.nn.one_hot(data["targets"], config.vocab_size)
loss, _ = max_utils.cross_entropy_with_logits(logits, one_hot_targets, 0.0)

print(f'loss shape: {loss.shape} mean: {loss.mean()}')
